뉴스 감성분석에 필요한 데이터 중, 부정 레이블로 분류된 기사들을 LDA(잠재 디리클레 할당) 분석기법을 통해 토픽별로 모델링한 결과입니다.

# 필요 패키지 로드

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 데이터 전처리

In [2]:
#데이터 불러오기
data1 = pd.read_csv('data_1121.csv', encoding = 'utf-8')

In [3]:
#데이터 정보 확인
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10977 entries, 0 to 10976
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10977 non-null  int64 
 1   index       10977 non-null  int64 
 2   icls_cnts   10977 non-null  object
 3   label       10977 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 343.2+ KB


In [4]:
#필요 레이블만 추출, value 별 확인
data1 = data1[['label','icls_cnts']]
data1['label'].value_counts()

 0    7681
 1    2087
-1    1209
Name: label, dtype: int64

In [5]:
#중복된 행 제거
data1.drop_duplicates(subset = ['icls_cnts'], inplace = True)
data1['label'].value_counts()

 0    6820
 1    2061
-1    1185
Name: label, dtype: int64

In [6]:
#부정 레이블만 추출
df1 = data1[data1['label']==-1]

In [7]:
#결측값 여부 확인
print('결측값 여부:',df1.isnull().values.any())

결측값 여부: False


In [8]:
import re 
#텍스트 정제 함수 : 한글 이외의 문자 제거
def text_cleaning(text):
    #한글 정규표현식으로 한글만 추출
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub(' ',text)
    return result

In [9]:
#함수 적용
df1['ko_text']=df1['icls_cnts'].apply(lambda x: text_cleaning(x))
df1.head

/tmp/ipykernel_259836/4292822634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ko_text']=df1['icls_cnts'].apply(lambda x: text_cleaning(x))


<bound method NDFrame.head of        label                                          icls_cnts  \
293       -1  ..<b>경찰</b> 지휘한다던 이상민 장관엔 왜 책임 안묻나” 일선 .. <b>경...   
453       -1  반면 국민의힘은 <b>경찰</b>의 늑장·부실대응을 참사의 근본적인 책임으로 지적했...   
478       -1  與 당권주자 안철수, 이상민 행정안전부 장관 자진사퇴 촉구 여당의 당권주자인 안철수...   
479       -1  ..<b>사고</b> 현장에서 가장 힘들었던 일은 .. 156명의 목숨을 앗아간 전...   
514       -1   8일 전국공무원노동조합 소방본부 서울소방지부(이하 서울소방노조)는 <b>경찰</b...   
...      ...                                                ...   
10465     -1  “<b>경찰</b> 출동을 독촉해 달라”는 요구가 소방 무전을 통해 전파된다...“...   
10478     -1  "서해 피격" 김홍희 .."서해 공무원 피살 사건"과 관련해 진상 은폐 의혹으로 구...   
10491     -1   이는 지난 7일 <b>경찰</b>청 특별<b>수사</b>본부(특수본)가 이임재 전...   
10571     -1  ..<b>경찰</b>청, 행정안전부, 국무총리실, 대통령실 등을 조.."신속한 강제...   
10960     -1   해당 보고서를 삭제하라는 지시가 더 윗선에서 내려왔다는 의혹에 대해서도 <b>수사...   

                                                 ko_text  
293     경찰 지휘한다던 이상민 장관엔 왜 책임 안묻나 일선 경찰 관이 아니라 국민의 입장...  
453    반면 국민의힘은 경찰 의 늑장 부실대응을 참사의 근본적인 책임으로 지적했

In [10]:
#pos tagging 함수
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

In [11]:
#상위 100개의 단어를 보존
#TF-IDF 행렬을 만들기 
vectorizer = TfidfVectorizer(max_features = 100)
X = vectorizer.fit_transform(df1['ko_text'])

#행렬 확인
print('TF-IDF 행렬의 크기:', X.shape)

TF-IDF 행렬의 크기: (1185, 100)


In [12]:
#LDA 토픽 모델링
lda_model = LatentDirichletAllocation(n_components =10, learning_method = 'online', random_state = 777, max_iter =1)
lda_top = lda_model.fit_transform(X)

print(lda_model.components_)
print(lda_model.components_.shape)

[[ 0.36096143  0.36528099  0.33754234  0.30172761  0.45456399  0.3034756
   0.9740398   0.46807593  0.38511848  5.11357089  0.29466476  0.28143405
   0.97159351  0.26727099  0.50137158  0.4025086   0.57120579  0.47951595
   0.31674734  0.26188438  0.26320284  0.57551543  1.26430923  0.46114486
   0.28863988  0.92841202  0.47087295  0.94638198  0.6674639   0.42638009
   1.11256255  0.26447646  0.31782775  0.4148092   0.53524328  0.27756669
   2.58137596  0.28426212  0.81000531  0.3655335   0.44602708  0.28909676
   1.60285799  0.48613878  0.84247354  0.33257615  1.55114666  0.92906998
   0.38847246  4.84976361  0.52229694  0.36091614  0.27555553  1.03230734
   0.53914782  1.60144904  0.28677737  0.42241381  0.37127205  0.29271556
   0.51051596  0.5506464   0.30542914  0.31270431  0.28908272  0.40968649
   0.71211412  0.85344135  0.2843896   0.27999827  0.95626329  0.32601236
   1.1412632   0.26823946  5.05596601  0.4400975   0.79220942  0.55295701
   0.6857881   0.54357741  0.31378999  

In [13]:
#토픽 추출 및 확인
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" %(idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:n -1:-1]])
        
get_topics(lda_model.components_,terms)

Topic 1: [('경찰', 5.11), ('지난해', 5.06), ('씨는', 4.85), ('범죄', 2.58), ('사고', 1.6), ('위해', 1.6), ('수사', 1.55), ('년간', 1.26), ('지난', 1.14), ('명이', 1.11), ('용산', 1.03), ('특별', 0.98), ('것으로', 0.97), ('경찰의', 0.97), ('조사를', 0.96), ('등을', 0.95), ('혐의로', 0.93), ('숨진', 0.93), ('대해', 0.93), ('정보계장', 0.85), ('서울', 0.84), ('특수본', 0.83), ('본부', 0.81), ('집회', 0.79), ('있다', 0.71), ('참사', 0.69), ('따르면', 0.67), ('최근', 0.65), ('핼러윈', 0.65), ('통해', 0.61), ('기자', 0.58), ('관련', 0.57), ('청장', 0.57), ('징계', 0.55), ('이태원', 0.55), ('책임을', 0.54), ('폭행', 0.54), ('위반', 0.54), ('받은', 0.54), ('씨를', 0.52), ('이번', 0.51), ('공무원', 0.5), ('서는', 0.49), ('피해', 0.48), ('관련해', 0.48), ('등에', 0.47), ('것은', 0.47), ('당시', 0.46), ('같은', 0.45), ('했다', 0.45), ('하는', 0.45), ('사건', 0.45), ('지적했다', 0.44), ('청은', 0.43), ('말했다', 0.43), ('의혹', 0.42), ('받는', 0.41), ('있는', 0.41), ('과정에서', 0.4), ('씨가', 0.39), ('현장', 0.39), ('경위를', 0.39), ('이날', 0.37), ('한다', 0.37), ('부실', 0.37), ('가장', 0.37), ('가운데', 0.36), ('앞서', 0.36), ('간부', 0.34), ('소속', 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
